<a href="https://colab.research.google.com/github/Kimyobu/Stable-diffusion-a1111-colab/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Install Stage**<br>
##**!!Important!!** Setup webUI first before run other cell
##**คำเตือน** ให้รัน Setup webUI ก่อนค่อยรัน cell อื่น



In [ ]:
#@markdown #**Setup webUI**<br>
#@markdown > Clone Stable diffusion webui git repo<br>* caution if instant_replace = True will destroy and replace webUI folder<br>* คำเตือนเปิด instant_replace จะลบ folder webUI แล้วลงใหม่ข้อมูลหายหมด
instant_replace = False #@param {type: "boolean"}
#@markdown
install_extension = True #@param {type: "boolean"}
#@markdown > Leave it default to install common extensions<br>ไม่ต้องเปลี่ยนถ้าจะลง extension ทั่วไป
extension_list_file = '/content/extensions.txt' #@param {type: "string"}
#@markdown
import_model_from_drive = True #@param {type: "boolean"}
import time
import os
import json
from google.colab import drive, output
def join(d: str, l: list[str], *a:str):
    for x in a:
        l.append(x)
    return d.join(l)

def rd(l):
    return [*set(l)]

def wait(t):
    time.sleep(t)

def mount():
    c = True
    p = '/content/drive'
    if os.path.exists(p):
        c = False
    if c :
        drive.mount(p, force_remount=True)

def git_clone(repo, path):
    if path:
        %cd {path}
    for x in repo:
        !git clone {x}
    %cd /content

def clear():
    output.clear()

def rm(path, arg = ''):
    if arg == '-r':
        os.removedirs(path)
    else:
        os.remove(path)

if import_model_from_drive:
   print('Please allow access to drive we will only save and read our service content')
   mount()

!apt-get -y install -qq aria2
!pip install --upgrade fastapi
!apt install libcairo2-dev pkg-config python3-dev
!pip install svglib
!pip install pytorch-lightning==1.6.5
!pip install torchmetrics==0.11.0
clear()
j = 'https://github.com/AUTOMATIC1111/stable-diffusion' + '-webui'
if instant_replace and os.path.exists('/content/project'):
  rm('/content/project', '-r')

!git clone {j} /content/project
if install_extension:
   !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Kimyobu/Stable-diffusion-a1111-colab/raw/main/SD/extensions.txt
   
   install = open(extension_list_file, 'r').read().split('\n')
   git_clone(install, '/content/project/extensions')
   print('\033[38;5;190mWait for install ControlNet Models\033[39m')
   p = '/content/project/extensions/sd-we' + ' bui-controlnet/models'
   !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Kimyobu/Stable-diffusion-a1111-colab/main/SD/control_net_model.json -d /content
   o = json.load(open('/content/control_net_model.txt', 'r'))
   for x in o:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {x} -o {o[x]} -d {p}

if import_model_from_drive:
   save_path =  '/content/drive/MyDrive/kimyobu-colab/models/'
   model_path = '/content/project/models/'
   !mkdir -p {save_path}
   for x in os.listdir(save_path):
       fol = model_path + x.split('.')[0]
       !mkdir -p {fol}
       !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -i {save_path + x} -d {fol}

os.environ['setup'] = 'True'
clear()
print('\033[38;5;82m' + 'Setup WebUI Complete')

In [ ]:
#@markdown #Check list of default common extension<br>ดู extension ทั่วไปที่จัดไว้ให้
!!apt-get -y install -qq aria2
!!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Kimyobu/Stable-diffusion-webui-colab/raw/main/SD/extensions.txt -d /content
print('\033[38;5;190mCurrent List:\n' + open('/content/extensions.txt').read())

---

#**Model downloader**<br>
## Put your model link file and select model type<br>
## ใส่ลิงค์ไฟล์โมเดลแล้วเลือกประเภทโมเดล<br>
> YumYumFusionMix CheckPoint <br>
https://civitai.com/models/99134/yumyumfusionmix<br>
https://civitai.com/api/download/models/113576<br>
YumYum2D<br>
https://civitai.com/models/106870/yumyum2d<br>
https://civitai.com/api/download/models/115417<br>
* @Credit <a href="https://civitai.com/user/AIJPG" title="Follower for support">AIJPG</a><br>
<a href="https://civitai.com/user/AIJPG"><img src="https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/20147aba-d01c-44b2-b8e0-c2040e46d436/width=128/AIJPG.jpeg" width="100" height="100" title="AIJPG"></a>

In [ ]:
#@title Runtime Install
#@markdown * Install to Runtime storage<br>อันนี้มันจะลงที่ Runtime
import os
if not os.environ.get('setup'):
    raise SystemExit('\033[31mRun SetUP WebUI FIRST!!\033[39m')
save_path = '/content/project/models/'
model_type = 'Stable-diffusion' #@param ["adetailer","deepdanbooru","karlo","sam","VAE-approx","Codeformer","ESRGAN","LDSR","Stable-diffusion","ControlNet","GFPGAN","Lora","SwinIR","deepbooru","hypernetworks","LyCORIS","VAE"]
model_path = save_path + model_type
model_url = 'https://civitai.com/api/download/models/113576' #@param {type:"string"}
#@markdown > Uninstall not support you can manually delete model file in models folder at /content/project/models<br>ไม่รองรับการลบโมเดลสามารถลบเองได้ที่ folder /content/project/models/
model_url = f"\"{model_url}\""
!mkdir -p "{model_path}"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {model_path}

In [ ]:
#@title Google_Drive Install
#@markdown * Install to Google_Drive storage<br>อันนี้มันจะลงที่ Google_Drive
import os
if not os.environ.get('setup'):
    raise SystemExit('\033[31mRun SetUP WebUI FIRST!!\033[39m')
mount()
check_list = False #@param {type: "boolean"}
save_path =  '/content/drive/MyDrive/kimyobu-colab/models/'
if check_list:
    for x in os.listdir(save_path):
        print(f"{x.split('.')[0]}: \n{open(x, 'r').read()}")
else :
    model_type = 'Stable-diffusion' #@param ["adetailer","deepdanbooru","karlo","sam","VAE-approx","Codeformer","ESRGAN","LDSR","Stable-diffusion","ControlNet","GFPGAN","Lora","SwinIR","deepbooru","hypernetworks","LyCORIS","VAE"]
    model_url = 'https://civitai.com/api/download/models/113576' #@param {type:"string"}
    #@markdown > Uninstall can manually edit at /content/drive/MyDrive/kimyobu-colab/models<br>สามารถลบโมเดลทที่เซฟไว้ในไดร์ฟที่ /content/drive/MyDrive/kimyobu-colab/models
    uninstall_model = False #@param {type: "boolean"}
    model_path = '/content/project/models/' + model_type
    !mkdir -p {save_path}
    path = save_path + model_type + '.txt'
    !touch {path}
    f = open(path, 'r')
    r = ''
    !rm -r {path}
    if uninstall_model:
        for x in rd(f.read().split('\n')):
            if x != model_url :
                r += x + '\n'
        f.close()
        open(path, 'a').write(r)
        print('\033[31mUninstall ' + model_url + ' \033[39mfrom \033[38;5;14m' + path)
    else :
        r = join('\n', rd(f.read().split('\n')), model_url)
        f.close()
        open(path, 'a').write(r)
        print('\n' + '\033[38;5;82mStatus OK ' + '\033[39mInstall model to \033[38;5;14m' + save_path + model_type)
        #Import state
        import_to_runtime = True #@param {type: "boolean"}
        if import_to_runtime and not uninstall_model:
            print('\n' + '\033[38;5;190mWait for install to runtime')
            !mkdir -p {model_path}
            model_url = f"\"{model_url}\""
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {model_path}
    print('\033[38;5;190mCurrent List:\n' + open(path, 'r').read())

---

#**Launch WebUI**

In [ ]:
#@markdown #**Launch WebUI**
import os
if not os.environ.get('setup'):
    raise SystemExit('\033[31mRun SetUP WebUI FIRST!!\033[39m')
%cd /content/project
#@markdown > Set to true for saveing outputs to google drive<br>เปิดไว้ถ้าจะให้เจนรูปไปไว้ใน google drive
save_to_drive = True #@param {type: "boolean"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Kimyobu/Stable-diffusion-a1111-colab/main/styles.csv
if save_to_drive:
   mount()
   !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Kimyobu/Stable-diffusion-a1111-colab/main/SD/config.json

clear()
# Web UI tunnel
!COMMANDLINE_ARGS="--share --no-half-vae --xformers --enable-insecure-extension- --theme dark" REQS_FILE="requirements.txt" python launch.py

In [ ]:
#@markdown * Copy generated files to drive
import os
if not os.environ.get('setup'):
    raise SystemExit('\033[31mRun SetUP WebUI FIRST!!\033[39m')
mount()
!cp -r /content/project/outputs/* /content/drive/MyDrive/kimyobu-colab/outputs

---

# @**Credit**
##**<a href="https://github.com/Kimyobu" title="Visit me on github">Kimyobu</a> <a href="https://discord.gg/aith" title="See us in community">Discord Community**</a>
<a href="https://github.com/Kimyobu" title="Visit me on github" style="margin-right: 100px;"><img src="https://avatars.githubusercontent.com/u/91616822?v=4" width=100px height=100px></a>  <a href="https://discord.gg/aith" title="See us in community"><img src="https://cdn.discordapp.com/icons/850018471096877096/a_79e2f1a6a99f763a924ef3e1d3ef7c6f.webp" width=100px height=100px></a>

---